In [ ]:
#commencer ici avec jupyter
# Initialisation du SparkContext
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [ ]:
#création de la fonction parseRatings
def parseRatings(row):
    splitted = row.split("::")
    return (int(splitted[0]), int(splitted[1]), int(splitted[2]), splitted[3])

In [ ]:
# 1. Créer le RDD « ratings » à partir du fichier « ratings.dat » et le mettre en cache.
ratings_rdd = sc.textFile("movieLens/ratings.dat").map(parseRatings).cache()

In [ ]:
# 1. Créer le RDD « ratings » à partir du fichier « ratings.dat » et le mettre en cache.
# (sans utilisation de fonction)
ratings_rdd2 = sc.textFile("movieLens/ratings.dat").map(lambda row: row.split("::")) \
    .map(lambda splitted: (int(splitted[0]), int(splitted[1]), int(splitted[2]), splitted[3])) \
    .cache()


In [ ]:
ratings_rdd.count()


1000209

In [ ]:
# 2. Combien de fois la note 1 a été attribuée ?
count_rating_1 = ratings_rdd.filter(lambda x: x[2] == 1).count()
print("Le nombre de fois où la note 1 a été attribuée est :", count_rating_1)

Le nombre de fois où la note 1 a été attribuée est : 56174


In [ ]:
ratings_rdd.take(5)

[(1, 1193, 5, '978300760'),
 (1, 661, 3, '978302109'),
 (1, 914, 3, '978301968'),
 (1, 3408, 4, '978300275'),
 (1, 2355, 5, '978824291')]

In [ ]:
# 3. Donner le nombre de films qui ont été notés (sans doublons).
unique_movies_rated = ratings_rdd.map(lambda x: x[1]).distinct().count()
print("Le nombre de films qui ont été notés est :", unique_movies_rated)

Le nombre de films qui ont été notés est : 3706


In [ ]:
# 4. Quel est l’utilisateur qui a donné le plus de notes. Donner le userID et le nombre de ses notes.
user_most_ratings = ratings_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).max(key=lambda x: x[1])
#user_most_ratings = ratings_rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda a, b: a + b).max(lambda x: x[1])
print("L'utilisateur ayant donné le plus de notes est UserID:", user_most_ratings[0], "avec", user_most_ratings[1], "notes.")


L'utilisateur ayant donné le plus de notes est UserID: 4169 avec 2314 notes.


In [ ]:
#5. Charger les fichiers movies.dat et users.dat et créer les RDD correspondants
movies_rdd = sc.textFile("movieLens/movies.dat").map(lambda row: row.split("::")).map(lambda splitted: (int(splitted[0]), splitted[1], splitted[2]))
users_rdd = sc.textFile("movieLens/users.dat").map(lambda row: row.split("::")).map(lambda splitted: (int(splitted[0]), splitted[1], int(splitted[2]), splitted[3], splitted[4]))
# 1. Créer le RDD « ratings » à partir du fichier « ratings.dat » et le mettre en cache.
#ratings_rdd = sc.textFile("movieLens/ratings.dat").map(parseRatings).cache()


In [ ]:
# 6. Donner la liste des genres triée et sans doublons.
distinct_genres = movies_rdd.flatMap(lambda x: x[2].split("|"))\
                                                    .distinct()\
                                                    .sortBy(lambda x: x)\
                                                    .collect()
print("Liste des genres triée et sans doublons :", distinct_genres)




Liste des genres triée et sans doublons : ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [ ]:
# 7. Donner le nombre de films de chaque genre.
movies_by_genre = movies_rdd.flatMap(lambda x: x[2].split("|"))\
                            .map(lambda genre: (genre, 1))\
                            .reduceByKey(lambda a, b: a + b)\
                            .collect()
print("Nombre de films de chaque genre :", movies_by_genre)


Nombre de films de chaque genre : [("Children's", 251), ('Fantasy', 68), ('Romance', 471), ('Drama', 1603), ('Action', 503), ('Thriller', 492), ('Horror', 343), ('Sci-Fi', 276), ('Documentary', 127), ('Musical', 114), ('Western', 68), ('Animation', 105), ('Comedy', 1200), ('Adventure', 283), ('Crime', 211), ('War', 143), ('Mystery', 106), ('Film-Noir', 44)]


In [ ]:
#8. Genres des films bien appréciés (note >= 4) par les hommes ayant un âge supérieur à 45
# Créer un RDD pour les films bien appréciés (note >= 4)
most_liked_movies_rdd = ratings_rdd.filter(lambda x: x[2]>= 4).map(lambda x: (x[0],x[1]))
# Créer un RDD pour les hommes matures
male45_users_rdd = users_rdd.filter(lambda x: x[1] == "M" and x[2] > 45).map(lambda x: (x[0],1))
# Joindre les deux RDD
joined_rdd = most_liked_movies_rdd.join(male45_users_rdd)
#joined_rdd.take(5)
#extraire les genrs des ces fillms bien aimées
genres_rdd = joined_rdd.map(lambda x: (x[1][0], x[1][1]))\
                      .join(movies_rdd.map(lambda x: (x[0], x[2])))\
                      .flatMap(lambda x: x[1][1].split("|"))\
                      .distinct()\
                      .sortBy(lambda x: x)
# Afficher les 20 premiers genres
print(genres_rdd.take(20))


In [ ]:
male45_users_rdd = users_rdd.filter(lambda x: x[1] == "M" and x[2] > 45)
male45_users_rdd.take(10)

In [ ]:
#9.Donner pour chaque année le genre le plus populaire des films
most_popular_genre_by_year = movies_rdd.map(lambda row: (row[1][-5:-1], row[2].split("|"))) \
    .flatMap(lambda x: map(lambda z: (x[0], z), x[1])) \
    .map(lambda x: ((x[0], x[1]), 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .map(lambda x: (x[0][0], (x[0][1], x[1]))) \
    .reduceByKey(lambda a, b: a if a[1] > b[1] else b) \
    .map(lambda x: (x[0], x[1][0])) \
    .sortBy(lambda x: x[0], ascending=False)

for result in most_popular_genre_by_year.collect():
    print(result)

In [ ]:
# 10. Pour les utilisateurs de moins de 18 ans (catégorie 1), quelle est la fréquence de chaque note ?
ratings_users_under_18 = ratings_rdd.map(lambda x: (x[0], x[2]))\
                                    .join(users_rdd.map(lambda x: (x[0], x[2])))\
                                    .filter(lambda x: x[1][1] < 18)\
                                    .map(lambda x: (x[1][0], 1))\
                                    .reduceByKey(lambda a, b: a + b)\
                                    .collect()
print("Fréquence de chaque note pour les utilisateurs de moins de 18 ans :", ratings_users_under_18)

Fréquence de chaque note pour les utilisateurs de moins de 18 ans : [(4, 8808), (5, 6802), (1, 2238), (2, 2983), (3, 6380)]
